In [6]:
#Understanding the data set
import pandas as pd

def inspect_x_test(file_path):
    try:
        df = pd.read_csv(file_path)
        print("Read with commas")
    except pd.errors.ParserError:
        try:
            df = pd.read_csv(file_path, delimiter='\t')
            print("Read with tabs")
        except Exception as e:
            print(f"Error reading the .txt file: {e}")
            return
    except Exception as e:
        print(f"Error reading the file: {e}")
        return

    print("Data Preview:")
    print(f"Number of rows (samples): {df.shape[0]}")
    print(f"Number of columns (features): {df.shape[1]}")
    print("\nColumn names:")
    print(df.columns.tolist())

# inspect_x_test("Data/x_test.txt")
# inspect_x_test("Data/y_test.txt")


def preview_large_csv(file_path, num_lines=20, delimiter=','):
    try:
        df = pd.read_csv(file_path, delimiter=delimiter, nrows=num_lines)
        print(df)
    except Exception as e:
        print(f"Error loading preview: {e}")

# preview_large_csv("Data/x_test.txt", delimiter='\t')

def check_nulls_in_data(x_file_path, y_file_path):
    def read_file_smart(path, label):
        try:
            df = pd.read_csv(path)
            print(f"{label}: Read with commas")
        except pd.errors.ParserError:
            try:
                df = pd.read_csv(path, delimiter='\t')
                print(f"{label}: Read with tabs")
            except Exception as e:
                print(f"Error reading the {label} file: {e}")
                return None
        except Exception as e:
            print(f"Error reading the {label} file: {e}")
            return None
        return df

    X = read_file_smart(x_file_path, "X")
    y = read_file_smart(y_file_path, "y")

    if X is None or y is None:
        return

    if X.isnull().values.any() or y.isnull().values.any():
        print("Null found")
    else:
        print("No nulls found")

check_nulls_in_data('Data/x_test.txt', 'Data/y_test.txt')
check_nulls_in_data('Data/x_train.txt', 'Data/y_train.txt')



X: Read with tabs
y: Read with commas
Null found
X: Read with tabs
y: Read with commas
Null found


In [ ]:
#Function to preprocess the data
def preprocessXandY(X, y):
    lang_codes = ['ita', 'fra', 'eng', 'ind', 'spa'] #lang codes of what we will use Italian, French, English, Indonesian, Spanish
    return None

